In [5]:
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline
from scipy import signal, optimize, stats
from observer import make_kfilter, vibe_fit_freq, get_psd, predict, update, noise_filter
from aberrations import make_1D_vibe_data

In [6]:
T = 1
f_sampling = 1000
steps = T * f_sampling
times = np.arange(0, steps/f_sampling, 1/f_sampling)

vibe = make_1D_vibe_data(N=5)
commands = np.zeros(steps)
centroid = np.zeros(steps+1)

for i in range(steps):
    if i >= 3:
        commands[i] = 0.6*commands[i-1] + 0.32*commands[i-2] + 0.08*commands[i-3] + 0.1*centroid[i]
    centroid[i+1] = vibe[i] - commands[i]

In [7]:
centroid_kalman = np.zeros(steps)

state, A, P, Q, H, R = make_kfilter(*vibe_fit_freq(noise_filter(get_psd(vibe)), N=10))
states_freq = np.zeros((steps, state.size))
for k in range(steps):
    state, P = update(H, P, R, state, vibe[k])
    states_freq[k] = state
    state, P = predict(A, P, Q, state)
    centroid_kalman[k] = vibe[k] - H.dot(A.dot(states_freq[k]))
    
pos_freq = np.array([H.dot(state) for state in states_freq]).flatten()

In [8]:
print("Average deviation in x, open loop: ", sum([np.mean(vibe**2)]))
print("Average deviation in x, standard integrator: ", sum([np.mean(centroid**2)]))
print("Average deviation in x, Kalman filter: ", sum([np.mean(pos_freq**2)]))

Average deviation in x, open loop:  0.6816061378268371
Average deviation in x, standard integrator:  0.685989358613586
Average deviation in x, Kalman filter:  0.5593753378177407
